# **DATA3888 Project: Optiver**

In [1]:
import warnings

import pandas as pd
import numpy as np
import polars as pl
from glob import glob

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression
from sklearn.metrics import r2_score, root_mean_squared_error

from scipy.stats import skew, pearsonr

In [2]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## **1. Combining CSVs**

In [3]:
csv_files = sorted(glob("Data/individual_book_train/*.csv"))

In [ ]:
ldf = pl.scan_csv(
    csv_files,
    schema_overrides={         
        'time_id': pl.Int32,
        'seconds_in_bucket': pl.Int32,
        'bid_price1': pl.Float32,
        'ask_price1': pl.Float32,
        'bid_price2': pl.Float32,
        'ask_price2': pl.Float32,
        'bid_size1': pl.Int32,
        'ask_size1': pl.Int32,
        'bid_size2': pl.Int32,
        'ask_size2': pl.Int32,
        'stock_id': pl.Int32,
    },
    infer_schema_length=0
)

df = ldf.collect()  
df.write_parquet("Data/112Stocks.parquet", compression="snappy")

In [4]:
df = pd.read_parquet("Data/112Stocks.parquet")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167253289 entries, 0 to 167253288
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   time_id            int32  
 1   seconds_in_bucket  int32  
 2   bid_price1         float32
 3   ask_price1         float32
 4   bid_price2         float32
 5   ask_price2         float32
 6   bid_size1          int32  
 7   ask_size1          int32  
 8   bid_size2          int32  
 9   ask_size2          int32  
 10  stock_id           int32  
dtypes: float32(4), int32(7)
memory usage: 6.9 GB


In [6]:
RANDOM_STATE = 42
VOLATILITY_IQR_MULTIPLIER = 1.5
N_CLUSTERS = 5 
MIN_PERIODS_FOR_MODEL = 10 
R2_WEIGHT = 0.5
QLIKE_WEIGHT = 0.5
EPSILON = 1e-9

In [7]:
def calculate_basic_features_snapshot(df_slice):
    features = pd.DataFrame(index=df_slice.index)
    features['micro_price'] = (df_slice['bid_price1'] * df_slice['ask_size1'] + \
                               df_slice['ask_price1'] * df_slice['bid_size1']) / \
                              (df_slice['bid_size1'] + df_slice['ask_size1'] + EPSILON)
    features['micro_price'] = features['micro_price'].fillna((df_slice['bid_price1'] + df_slice['ask_price1']) / 2)
    features['spread1'] = df_slice['ask_price1'] - df_slice['bid_price1']
    features['spread2'] = df_slice['ask_price2'] - df_slice['bid_price2']
    features['imbalance_size1'] = (df_slice['bid_size1'] - df_slice['ask_size1']) / \
                                  (df_slice['bid_size1'] + df_slice['ask_size1'] + EPSILON)
    sum_bid_sizes = df_slice['bid_size1'] + df_slice['bid_size2']
    sum_ask_sizes = df_slice['ask_size1'] + df_slice['ask_size2']
    features['book_pressure'] = sum_bid_sizes / (sum_bid_sizes + sum_ask_sizes + EPSILON)
    return features

def calculate_time_id_features(df_group):
    df_group = df_group.sort_values('seconds_in_bucket').copy() # Ensure correct order for log returns
    snapshot_features = calculate_basic_features_snapshot(df_group)
    log_returns = np.log(snapshot_features['micro_price'] / snapshot_features['micro_price'].shift(1))
    log_returns = log_returns.replace([np.inf, -np.inf], np.nan).dropna()
    results = {}
    results['realized_volatility'] = np.std(log_returns) if len(log_returns) > 1 else 0
    results['realized_skewness'] = skew(log_returns) if len(log_returns) > 1 else 0
    if len(log_returns) > 2: 
        ac, _ = pearsonr(log_returns.iloc[1:], log_returns.iloc[:-1])
        results['autocorrelation_log_returns'] = ac if not np.isnan(ac) else 0
    else:
        results['autocorrelation_log_returns'] = 0
        
    results['tick_frequency'] = len(df_group)
    results['mean_micro_price'] = snapshot_features['micro_price'].mean()
    results['mean_spread1'] = snapshot_features['spread1'].mean()
    results['mean_spread2'] = snapshot_features['spread2'].mean()
    results['mean_imbalance_size1'] = snapshot_features['imbalance_size1'].mean()
    results['mean_book_pressure'] = snapshot_features['book_pressure'].mean()
    results['mean_bid_size1'] = df_group['bid_size1'].mean()
    results['mean_ask_size1'] = df_group['ask_size1'].mean()
    results['mean_bid_size2'] = df_group['bid_size2'].mean()
    results['mean_ask_size2'] = df_group['ask_size2'].mean()
            
    return pd.Series(results)

def qlike_loss(y_true, y_pred):
    y_pred = np.maximum(y_pred, EPSILON) 
    y_true = np.maximum(y_true, 0)      
    valid_indices = (y_true > EPSILON) 
    if not np.any(valid_indices):
        return np.nan 
    y_true_f = y_true[valid_indices]
    y_pred_f = y_pred[valid_indices]
    y_pred_f = np.maximum(y_pred_f, EPSILON)
    loss = np.mean(y_true_f / y_pred_f - np.log(y_true_f / y_pred_f) - 1)
    return loss

In [8]:
print("Calculating features per stock_id and time_id...")
stock_time_id_features = df.groupby(['stock_id', 'time_id']).apply(calculate_time_id_features).reset_index()
print(f"Calculated detailed features for {stock_time_id_features.shape[0]} stock/time_id pairs.")
print(stock_time_id_features.head())

Calculating features per stock_id and time_id...
Calculated detailed features for 428932 stock/time_id pairs.
   stock_id  time_id  realized_volatility  realized_skewness  \
0         0        5             0.000259           0.331051   
1         0       11             0.000085          -1.095674   
2         0       16             0.000173          -0.190956   
3         0       31             0.000235          -1.939494   
4         0       62             0.000143           0.450891   

   autocorrelation_log_returns  tick_frequency  mean_micro_price  \
0                    -0.229365           302.0          1.003725   
1                    -0.223872           200.0          1.000239   
2                    -0.315257           188.0          0.999542   
3                    -0.114241           120.0          0.998832   
4                    -0.217718           176.0          0.999619   

   mean_spread1  mean_spread2  mean_imbalance_size1  mean_book_pressure  \
0      0.000855      

In [9]:
print("\nFiltering pathological stocks...")
overall_stock_mean_rv = stock_time_id_features.groupby('stock_id')['realized_volatility'].mean().reset_index()
overall_stock_mean_rv = overall_stock_mean_rv.rename(columns={'realized_volatility': 'mean_realized_volatility'})

q1 = overall_stock_mean_rv['mean_realized_volatility'].quantile(0.25)
q3 = overall_stock_mean_rv['mean_realized_volatility'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - VOLATILITY_IQR_MULTIPLIER * iqr
upper_bound = q3 + VOLATILITY_IQR_MULTIPLIER * iqr

epsilon_vol = 1e-7 
filtered_stocks_info = overall_stock_mean_rv[
    (overall_stock_mean_rv['mean_realized_volatility'] >= lower_bound) &
    (overall_stock_mean_rv['mean_realized_volatility'] <= upper_bound) &
    (overall_stock_mean_rv['mean_realized_volatility'] > epsilon_vol)
]

n_original_stocks = df['stock_id'].nunique()
n_filtered_stocks = filtered_stocks_info['stock_id'].nunique()
print(f"Original number of stocks: {n_original_stocks}")
print(f"Number of stocks after volatility filtering: {n_filtered_stocks}")

if n_filtered_stocks == 0:
    print("Error: No stocks remaining after filtering. Adjust VOLATILITY_IQR_MULTIPLIER or check data.")


Filtering pathological stocks...
Original number of stocks: 112
Number of stocks after volatility filtering: 111


In [10]:
stock_time_id_features_filtered = stock_time_id_features[
    stock_time_id_features['stock_id'].isin(filtered_stocks_info['stock_id'])
]

In [11]:
print("\nEngineering features for K-means clustering...")

cluster_feature_cols = [
    'realized_volatility', 'realized_skewness', 'autocorrelation_log_returns', 
    'tick_frequency', 'mean_micro_price', 'mean_spread1', 'mean_spread2', 
    'mean_imbalance_size1', 'mean_book_pressure',
    'mean_bid_size1', 'mean_ask_size1', 'mean_bid_size2', 'mean_ask_size2'
]

stock_meta_features_df = stock_time_id_features_filtered.groupby('stock_id')[cluster_feature_cols].mean()
print("Meta-features for clustering (mean of time_id features per stock):")
print(stock_meta_features_df.head())


Engineering features for K-means clustering...
Meta-features for clustering (mean of time_id features per stock):
          realized_volatility  realized_skewness  autocorrelation_log_returns  \
stock_id                                                                        
0                    0.000291          -0.029187                    -0.189381   
1                    0.000242          -0.017519                    -0.154757   
2                    0.000117          -0.048594                    -0.138243   
3                    0.000374          -0.037155                    -0.173425   
4                    0.000276           0.047078                    -0.191476   

          tick_frequency  mean_micro_price  mean_spread1  mean_spread2  \
stock_id                                                                 
0             239.569974          1.000072      0.001033      0.001432   
1             393.611488          1.000007      0.000708      0.001061   
2             496.903

In [12]:
scaler = StandardScaler()
scaled_meta_features = scaler.fit_transform(stock_meta_features_df)

In [13]:
print(f"\nPerforming K-means clustering with K={N_CLUSTERS}...")
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE, n_init='auto')
stock_meta_features_df['cluster'] = kmeans.fit_predict(scaled_meta_features)
print("Clustering results (stock_id and assigned cluster):")
print(stock_meta_features_df[['cluster']].head())


Performing K-means clustering with K=5...
Clustering results (stock_id and assigned cluster):
          cluster
stock_id         
0               3
1               2
2               2
3               4
4               3


In [14]:
print("\nCalculating R-squared and QLIKE scores using expanding window approach...")
r_squared_feature_cols = [
    'realized_volatility', 'mean_spread1', 'mean_imbalance_size1', 
    'mean_book_pressure', 'mean_micro_price'
]
stock_scores_list = []
stock_time_id_features_filtered = stock_time_id_features_filtered.sort_values(['stock_id', 'time_id'])


Calculating R-squared and QLIKE scores using expanding window approach...


In [15]:
for stock_id in filtered_stocks_info['stock_id']:
    stock_data = stock_time_id_features_filtered[stock_time_id_features_filtered['stock_id'] == stock_id].copy()
    
    if len(stock_data) < MIN_PERIODS_FOR_MODEL:
        print(f"Stock {stock_id}: Insufficient data ({len(stock_data)} periods) for R2/QLIKE, skipping.")
        stock_scores_list.append({'stock_id': stock_id, 'r_squared': np.nan, 'qlike': np.nan})
        continue

    for col in r_squared_feature_cols:
        stock_data[f'prev_{col}'] = stock_data[col].shift(1)
    
    stock_data = stock_data.dropna() 

    if len(stock_data) < 2: 
        print(f"Stock {stock_id}: Insufficient data after lagging for R2/QLIKE, skipping.")
        stock_scores_list.append({'stock_id': stock_id, 'r_squared': np.nan, 'qlike': np.nan})
        continue

    y_true_r2_all = []
    y_pred_r2_all = []
    y_true_qlike_all = []
    y_pred_qlike_all = []

    start_prediction_idx = max(2, MIN_PERIODS_FOR_MODEL // 2)


    for i in range(start_prediction_idx, len(stock_data)):
        train_df = stock_data.iloc[:i]
        current_period_data = stock_data.iloc[i]

        X_train = train_df[[f'prev_{col}' for col in r_squared_feature_cols]]
        y_train = train_df['realized_volatility']
        
        X_current = pd.DataFrame(current_period_data[[f'prev_{col}' for col in r_squared_feature_cols]]).T
        y_current_true_r2 = current_period_data['realized_volatility']

        if len(X_train) >= 2: 
            try:
                model = LinearRegression()
                model.fit(X_train, y_train)
                y_current_pred_r2 = model.predict(X_current)[0]
                
                y_true_r2_all.append(y_current_true_r2)
                y_pred_r2_all.append(y_current_pred_r2)
            except Exception:
                pass

        historical_rv_for_qlike = train_df['realized_volatility']
        if not historical_rv_for_qlike.empty:
            forecast_rv_qlike = historical_rv_for_qlike.mean()
            y_current_true_qlike = current_period_data['realized_volatility']

            y_true_qlike_all.append(y_current_true_qlike)
            y_pred_qlike_all.append(forecast_rv_qlike)

    r_squared_stock = np.nan
    if len(y_true_r2_all) >= 2 and len(set(y_true_r2_all)) > 1: 
        r_squared_stock = r2_score(y_true_r2_all, y_pred_r2_all)
    
    qlike_stock = np.nan
    if y_true_qlike_all:
        qlike_stock = qlike_loss(np.array(y_true_qlike_all), np.array(y_pred_qlike_all))

    stock_scores_list.append({
        'stock_id': stock_id,
        'r_squared': r_squared_stock,
        'qlike': qlike_stock
    })
    print(f"Stock {stock_id}: R^2 = {r_squared_stock:.4f}, QLIKE = {qlike_stock:.4f} (from {len(y_true_r2_all)} R2 points, {len(y_true_qlike_all)} QLIKE points)")

Stock 0: R^2 = -0.0167, QLIKE = 0.2002 (from 3824 R2 points, 3824 QLIKE points)
Stock 1: R^2 = -0.0256, QLIKE = 0.1533 (from 3824 R2 points, 3824 QLIKE points)
Stock 2: R^2 = -0.0286, QLIKE = 0.2506 (from 3824 R2 points, 3824 QLIKE points)
Stock 3: R^2 = -0.2260, QLIKE = 0.1261 (from 3824 R2 points, 3824 QLIKE points)
Stock 4: R^2 = -0.0602, QLIKE = 0.1870 (from 3824 R2 points, 3824 QLIKE points)
Stock 5: R^2 = -0.0390, QLIKE = 0.1365 (from 3824 R2 points, 3824 QLIKE points)
Stock 6: R^2 = -0.0922, QLIKE = 0.0973 (from 3824 R2 points, 3824 QLIKE points)
Stock 7: R^2 = -0.0177, QLIKE = 0.1046 (from 3824 R2 points, 3824 QLIKE points)
Stock 8: R^2 = -0.0135, QLIKE = 0.1477 (from 3824 R2 points, 3824 QLIKE points)
Stock 9: R^2 = -0.0932, QLIKE = 0.1539 (from 3824 R2 points, 3824 QLIKE points)
Stock 10: R^2 = -0.0263, QLIKE = 0.1909 (from 3824 R2 points, 3824 QLIKE points)
Stock 11: R^2 = -0.0585, QLIKE = 0.1841 (from 3824 R2 points, 3824 QLIKE points)
Stock 13: R^2 = -0.0534, QLIKE = 0.199

In [16]:
stock_scores_df = pd.DataFrame(stock_scores_list)
stock_scores_df = pd.merge(stock_scores_df, stock_meta_features_df[['cluster']].reset_index(), on='stock_id', how='left')
print("\nCalculated R-squared and QLIKE scores:")
print(stock_scores_df.head())


Calculated R-squared and QLIKE scores:
   stock_id  r_squared     qlike  cluster
0         0  -0.016697  0.200236        3
1         1  -0.025567  0.153346        2
2         2  -0.028575  0.250551        2
3         3  -0.225981  0.126084        4
4         4  -0.060184  0.186953        3


In [17]:
print("\nCombining scores and selecting top stocks...")
stock_scores_df = stock_scores_df.dropna(subset=['r_squared', 'qlike'])
if stock_scores_df.empty:
    print("Error: No stocks remaining after calculating R-squared/QLIKE (all NaNs or empty). Check calculation steps or MIN_PERIODS_FOR_MODEL.")
    exit()


Combining scores and selecting top stocks...


In [18]:
stock_scores_df['combined_score'] = R2_WEIGHT * stock_scores_df['r_squared'] - \
                                    QLIKE_WEIGHT * stock_scores_df['qlike']
top_stocks = stock_scores_df.sort_values(by='combined_score', ascending=False)
print(f"\nTop 30 stocks based on combined score ({R2_WEIGHT}*R^2 - {QLIKE_WEIGHT}*QLIKE):")
N_TOP_STOCKS = 30
final_selection = top_stocks.head(N_TOP_STOCKS)
print(final_selection)


Top 30 stocks based on combined score (0.5*R^2 - 0.5*QLIKE):
     stock_id  r_squared     qlike  cluster  combined_score
7           7  -0.017700  0.104596        2       -0.061148
83         96  -0.013730  0.112310        2       -0.063020
70         81  -0.009333  0.122045        1       -0.065689
44         50  -0.012519  0.123174        0       -0.067847
55         63  -0.021452  0.134259        2       -0.077856
48         55  -0.017104  0.141370        4       -0.079237
73         84  -0.015184  0.146063        2       -0.080624
8           8  -0.013549  0.147736        2       -0.080642
68         78  -0.014410  0.147396        4       -0.080903
63         73  -0.037662  0.129327        1       -0.083494
75         86  -0.030463  0.138283        1       -0.084373
66         76  -0.015786  0.153200        2       -0.084493
84         97  -0.037139  0.131908        4       -0.084523
20         22  -0.012541  0.156816        4       -0.084679
105       120  -0.010393  0.159367    

In [7]:
selected_stock_ids = [1, 5, 7, 8, 22, 27, 32, 44, 50, 55, 
                      59, 62, 63, 73, 75, 76, 78, 80, 81, 84, 
                      85, 86, 89, 96, 97, 101, 102, 109, 115, 120]

In [ ]:
df = pd.read_parquet("Data/112Stocks.parquet")
df = df[df["stock_id"].isin(selected_stock_ids)]
df.to_parquet("Data/30Stocks.parquet")

## **Feature Engineering and Selection**

In [2]:
df = pd.read_parquet("Data/30stocks.parquet")

In [3]:
def make_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['mid_price'] = (df['bid_price1'] + df['ask_price1']) / 2
    df['spread']    = df['ask_price1'] - df['bid_price1']
    
    with np.errstate(divide='ignore', invalid='ignore'):
        num  = df['bid_size1'] - df['ask_size1']
        den  = df['bid_size1'] + df['ask_size1']
        df['imbalance'] = np.where(den > 0, num / den, np.nan)

        num2 = (df['bid_size1'] + df['bid_size2']) - (df['ask_size1'] + df['ask_size2'])
        den2 = df[['bid_size1','bid_size2','ask_size1','ask_size2']].sum(axis=1)
        df['book_pressure'] = np.where(den2 > 0, num2 / den2, np.nan)

    df['normalized_spread'] = df['spread'] / df['mid_price'].replace(0, np.nan)
    df['OBI_L2'] = np.where(den2 > 0, (df['bid_size1'] + df['bid_size2']) / den2, np.nan)

    sizes = df[['bid_size1','bid_size2','ask_size1','ask_size2']].astype(float).values
    total = sizes.sum(axis=1, keepdims=True)
    p = np.divide(sizes, total, where=total != 0)
    entropy = -np.nansum(np.where(p > 0, p * np.log(p), 0), axis=1)
    df['LOB_entropy'] = entropy
    df['LOB_entropy_normalized'] = entropy / np.log(4)

    df['log_return'] = (
        df.groupby('time_id')['mid_price']
          .transform(lambda x: np.log(x / x.shift(1)))
    )

    df['realized_volatility'] = (
        df.groupby('time_id')['log_return']
        .transform(lambda x: np.sqrt(
            ((x.shift(1) ** 2)
                .rolling(30, min_periods=1)
                .sum()
            ).clip(lower=0)
        ))
    )

    df['rv_future'] = (
        df.groupby('time_id')['realized_volatility'].shift(-30)   
    )

    df['bipower_var'] = (
        df.groupby('time_id')['log_return']
          .transform(lambda x: x.abs().shift(1)
                       .rolling(2, min_periods=1)
                       .apply(lambda r: r[0] * r[1], raw=True)
                       .rolling(30, min_periods=1)
                       .mean())
    )

    df['wap'] = (
        (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) /
        (df['bid_size1'] + df['ask_size1']).replace(0, np.nan)
    )
    
    df['log_wap_return'] = (
        df.groupby('time_id')['wap']
          .transform(lambda x: np.log(x / x.shift(1)))
    )

    for col in ['imbalance', 'book_pressure', 'log_return']:
        df[f'{col}_lag1'] = df.groupby('time_id')[col].shift(1)
        df[f'{col}_lag2'] = df.groupby('time_id')[col].shift(2)

    df['rolling_vol_30'] = (
        df.groupby('time_id')['log_return']
          .transform(lambda x: x.shift(1).rolling(30, min_periods=1).std())
    )
    df['rolling_imbalance_mean_30'] = (
        df.groupby('time_id')['imbalance']
          .transform(lambda x: x.shift(1).rolling(30, min_periods=1).mean())
    )

    df = df.dropna()   
    df = df.replace([np.inf, -np.inf], np.nan)

    theta = 2 * np.pi * df['seconds_in_bucket'] / 600 # period = 600
    df['sec_sin'] = np.sin(theta)
    df['sec_cos'] = np.cos(theta)

    for c in ['bid_size1','ask_size1','bid_size2','ask_size2']:
        df[c + '_log'] = np.log1p(df[c])
        df.drop(columns=c, inplace=True)

    return df

In [4]:
df = make_features(df)

In [5]:
df = df.astype({col: 'float32' if df[col].dtype == 'float64' else 'int32' 
                for col in df.columns 
                if df[col].dtype in ['float64', 'int64']})

In [6]:
X = df.drop(columns=['rv_future'])

In [7]:
selector = VarianceThreshold(threshold=0.0)
X_reduced = selector.fit_transform(X)
selected_columns = X.columns[selector.get_support()]
X_reduced_df = pd.DataFrame(X_reduced, columns=selected_columns, index=X.index)

In [8]:
dfR = X_reduced_df.astype({col: 'float32' if X_reduced_df[col].dtype == 'float64' else 'int32' 
                for col in X_reduced_df.columns 
                if X_reduced_df[col].dtype in ['float64', 'int64']})

In [9]:
corr = dfR.corr(method='spearman').abs()
to_drop = {c for c in corr.columns for r in corr.columns
if r != c and corr.loc[r, c] > .98 and corr.loc[r].sum() < corr.loc[c].sum()}
to_drop

{'LOB_entropy_normalized',
 'OBI_L2',
 'ask_price1',
 'ask_price2',
 'bid_price1',
 'bid_price2',
 'normalized_spread',
 'realized_volatility',
 'wap'}

In [10]:
dfR = dfR.drop(columns=list(to_drop))

In [11]:
dfR['rv_future'] = df['rv_future']

In [12]:
dfR = dfR.sort_values(
    ['stock_id', 'time_id', 'seconds_in_bucket'], # Assuming they are not removed.
    ascending=[True, True, True]
).reset_index(drop=True)

In [13]:
dfR.to_parquet("Data/FE30Stocks.parquet")